In [1]:
import pandas as pd
import glob
import networkx as nx
from functions import *
import os
import scipy as sp
import scipy.io  
import io
import snap

In [2]:
path = "less_than_1500_nodes"
[x.replace("{}\\".format(path), "") for x in glob.glob("{}/*".format(path))]

['7th_graders',
 'aves-weaver-social',
 'bt.csv',
 'contacts-prox-high-school-2013',
 'contacts-prox-high-school-2013-attr',
 'email_company',
 'fb-messages',
 'fb-pages-politician',
 'fb-pages-tvshow',
 'fb_friends.csv',
 'high_tech_company',
 'inf-euroroad',
 'infect-dublin',
 'infect-hyper',
 'Karate',
 'kidnappings',
 'KKI',
 'kor.2015.mers.1.00',
 'law_firm',
 'moreno_taro',
 'physician_trust',
 'primary-school-proximity',
 'sgp.2003.sars.1.00',
 'soc-firm-hi-tech',
 'soc-hamsterster',
 'swingers',
 'uni_email',
 'usa.2009.flu.1.00',
 'usa.2020.covid.6.00',
 'webkb-wisc']

In [3]:
data_paths = [x.replace("{}\\".format(path), "") for x in glob.glob("{}/*".format(path))]

In [4]:
G_list = []
for d_path in data_paths:
    print(d_path)
    if any(fname.endswith('.csv') for fname in  glob.glob('{}/{}/*'.format(path,d_path))):
        df = pd.read_csv('{}/{}/edges.csv'.format(path,d_path))
        G = nx.from_pandas_edgelist(df, "# source", " target")
        
    elif any(fname.endswith('.edgelist') for fname in glob.glob('{}/{}/*'.format(path,d_path))):
        G = nx.read_edgelist("{}/{}/edges.edgelist".format(path, d_path), nodetype=int)
        
    elif not any(fname.endswith('txt.gz') for fname in  glob.glob('{}/{}/*'.format(path,d_path))):
        G = nx.read_adjlist("{}/{}/edges.txt".format(path, d_path), nodetype=int)

    else:
        G = nx.read_adjlist(gzip.open('{}/{}/edges.txt.gz'.format(path,d_path)), nodetype=int)
            
    G_list.append(G)

7th_graders
aves-weaver-social
bt.csv
contacts-prox-high-school-2013
contacts-prox-high-school-2013-attr
email_company
fb-messages
fb-pages-politician
fb-pages-tvshow
fb_friends.csv
high_tech_company
inf-euroroad
infect-dublin
infect-hyper
Karate
kidnappings
KKI
kor.2015.mers.1.00
law_firm
moreno_taro
physician_trust
primary-school-proximity
sgp.2003.sars.1.00
soc-firm-hi-tech
soc-hamsterster
swingers
uni_email
usa.2009.flu.1.00
usa.2020.covid.6.00
webkb-wisc


In [5]:
G_list

In [6]:
%%time
# Parameters 
nsamp = 10
p = 0.05
big_dat = pd.DataFrame({"seeding method": ['random']*nsamp + ['friend']*nsamp + ['pair']*nsamp})

for i,G in enumerate(G_list):    
    try:
        print(data_paths[i])
        print(nx.number_of_nodes(G_list[i]))

        (rtran, ftran, ptran) = sim_2seed_transitivity(G_list[i], p = p, nsamp = nsamp)
        # append results as columns
        big_dat["{}_num_nodes:{}".format(data_paths[i],  nx.number_of_nodes(G_list[i]))] = rtran + ftran + ptran
    except:
        print(data_paths[i])
    

7th_graders
29
aves-weaver-social
445
bt.csv
692
bt.csv
contacts-prox-high-school-2013
327
contacts-prox-high-school-2013-attr
327
email_company
167
fb-messages
1899
fb-pages-politician
5908
fb-pages-tvshow
3892
fb_friends.csv
800
high_tech_company
21
inf-euroroad
1174
infect-dublin
410
infect-hyper
113
Karate
34
kidnappings
351
kidnappings
KKI
2238
kor.2015.mers.1.00
186
kor.2015.mers.1.00
law_firm
71
moreno_taro
22
physician_trust
241
primary-school-proximity
242
sgp.2003.sars.1.00
172
sgp.2003.sars.1.00
soc-firm-hi-tech
33
soc-hamsterster
2426
swingers
96
swingers
uni_email
1133
usa.2009.flu.1.00
286
usa.2009.flu.1.00
usa.2020.covid.6.00
92
usa.2020.covid.6.00
webkb-wisc
265
Wall time: 3min 27s


In [7]:
big_dat.to_csv("2seedTransivityData1.csv")

In [8]:
big_dat

,seeding method,7th_graders_num_nodes:29,aves-weaver-social_num_nodes:445,contacts-prox-high-school-2013_num_nodes:327,contacts-prox-high-school-2013-attr_num_nodes:327,email_company_num_nodes:167,fb-messages_num_nodes:1899,fb-pages-politician_num_nodes:5908,fb-pages-tvshow_num_nodes:3892,fb_friends.csv_num_nodes:800,...,Karate_num_nodes:34,KKI_num_nodes:2238,law_firm_num_nodes:71,moreno_taro_num_nodes:22,physician_trust_num_nodes:241,primary-school-proximity_num_nodes:242,soc-firm-hi-tech_num_nodes:33,soc-hamsterster_num_nodes:2426,uni_email_num_nodes:1133,webkb-wisc_num_nodes:265
0,random,0.0,0.001150,0.000563,0.002165,0.002667,0.001496,0.000827,0.001189,0.001017,...,0.0,0.000784,0.000947,0.0,0.001121,0.001211,0.000000,0.000372,0.000228,0.000000
1,random,0.0,0.000000,0.000563,0.001251,0.000718,0.001265,0.000657,0.002453,0.000660,...,0.0,0.000000,0.000229,0.0,0.000498,0.001030,0.000000,0.000514,0.000301,0.000331
2,random,0.0,0.001342,0.000909,0.000983,0.001692,0.000841,0.001418,0.000301,0.000470,...,0.0,0.001456,0.000425,0.0,0.000249,0.000626,0.000000,0.000717,0.000353,0.000441
3,random,0.0,0.004026,0.000641,0.001134,0.001110,0.001036,0.000688,0.001813,0.000595,...,0.0,0.001344,0.001306,0.0,0.000249,0.000815,0.000000,0.000381,0.000498,0.000221
4,random,0.0,0.001534,0.000610,0.000987,0.000344,0.001315,0.000988,0.002365,0.000375,...,0.0,0.000280,0.000588,0.0,0.000000,0.000723,0.000000,0.000891,0.000259,0.000221
5,random,0.0,0.000192,0.000719,0.000571,0.001144,0.001205,0.001276,0.002586,0.000363,...,0.0,0.001120,0.000523,0.0,0.000000,0.000852,0.000000,0.000865,0.000394,0.000000
6,random,0.0,0.001534,0.001532,0.001446,0.002080,0.000836,0.000785,0.000626,0.000743,...,0.0,0.001232,0.001339,0.0,0.000125,0.001133,0.000000,0.000666,0.000280,0.000000
7,random,0.0,0.001246,0.001762,0.001069,0.001125,0.000884,0.000750,0.000888,0.000559,...,0.0,0.000896,0.000816,0.0,0.000748,0.001253,0.000000,0.000384,0.000104,0.000662
8,random,0.0,0.003547,0.002048,0.000494,0.000577,0.000905,0.000934,0.001569,0.000333,...,0.0,0.000672,0.000000,0.0,0.000249,0.001023,0.000000,0.000219,0.000218,0.000441
9,random,0.0,0.000863,0.000870,0.000887,0.000790,0.000823,0.000488,0.000945,0.000464,...,0.0,0.002408,0.001568,0.0,0.000125,0.001159,0.000000,0.000492,0.000259,0.000221


In [9]:
G_random = []
for i, G_empirical in enumerate(G_list):
    try:
        print(data_paths[i])
        G = nx.barabasi_albert_graph(len(G_empirical.nodes()), int(np.average(G_empirical.degree())/2))
        G_random.append(G)
    except:
        G_random.append(None)
        print(data_paths[i])

7th_graders
aves-weaver-social
bt.csv
contacts-prox-high-school-2013
contacts-prox-high-school-2013-attr
email_company
fb-messages
fb-pages-politician
fb-pages-tvshow
fb_friends.csv
high_tech_company
inf-euroroad
infect-dublin
infect-hyper
Karate
kidnappings
KKI
kor.2015.mers.1.00
kor.2015.mers.1.00
law_firm
moreno_taro
physician_trust
primary-school-proximity
sgp.2003.sars.1.00
sgp.2003.sars.1.00
soc-firm-hi-tech
soc-hamsterster
swingers
uni_email
usa.2009.flu.1.00
usa.2009.flu.1.00
usa.2020.covid.6.00
usa.2020.covid.6.00
webkb-wisc


In [ ]:
%%time
# Parameters 
nsamp = 10
p = 0.05
big_dat_random = pd.DataFrame({"seeding method": ['random']*nsamp + ['friend']*nsamp + ['pair']*nsamp})

for i,G in enumerate(G_random):    
    try:
        print(data_paths[i])
        print(nx.number_of_nodes(G_random[i]))

        (rtran, ftran, ptran) = sim_2seed_transitivity(G_random[i], p = p, nsamp = nsamp)
        # append results as columns
        big_dat_random["{}_num_nodes:{}".format(data_paths[i],  nx.number_of_nodes(G_random[i]))] = rtran + ftran + ptran
    except:
        print(data_paths[i])

7th_graders
29
aves-weaver-social
445
bt.csv
692
bt.csv
contacts-prox-high-school-2013
327
contacts-prox-high-school-2013-attr
327
email_company
167
fb-messages
1899


In [66]:
big_dat_random.to_csv("2seedTransivityDataRandom1.csv")

In [63]:
len(G_random[6].edges())

399578

In [64]:
len(G_list[6].edges())

15737